In [1]:
import numpy as np
import pandas as pd
map = {
    'G':0,
    'A':0,
    'V':0,
    'L':0,
    'I':0,
    'S':1,
    'C':1,
    'T':1,
    'M':1,
    'P':2,
    'F':3,
    'Y':3,
    'W':3,
    'H':4,
    'K':4,
    'R':4,
    'D':5,
    'E':5,
    'N':5,
    'Q':5
}
def replace(a1, a2):
    global map
    if a1 == a2:
        return 0
    if a1 == 'X' or a2 == 'X':
        return 1
    else:
        return 1 if map[a1] == map[a2] else 2
        
def levenshtein(seq1, seq2):
    if len(seq1) < len(seq2):
        return levenshtein(seq2, seq1)

    if len(seq2) == 0:
        return len(seq1)
    
    memo = np.zeros((len(seq1) +1, len(seq2) + 1), np.int64)

    memo[0] = range(len(seq2) + 1)
    
    for i, a1 in enumerate(seq1):
        memo[i + 1, 0] = i + 1
        for j, a2 in enumerate(seq2):
            insertions = memo[i, j + 1] + 1
            deletions = memo[i + 1, j] + 1
            substitutions = memo[i, j] + replace(a1, a2)
            memo[i + 1, j + 1] = min(insertions, deletions, substitutions)    
    return memo[-1,-1]

## Step 3.1

In [2]:
seq1 = ['M','N','I','K','G','S','P','W','K','G','S','L','L','L','L','L','V','S','N','L','L',
        'L','C','Q','S','V','A','P','L','P','I','C','P','G','G','A','A','R','C','Q','V','T',
        'L','R','D','L','F','D','R','A','V','V','L','S','H','Y','I','H','N','L','S','S','E',
        'M','F','S','E','F','D','K','R','Y','T','H','G','R','G','F','I','T','K','A','I','N',
        'S','C','H','T','S','S','L','A','T','P','E','D','K','E','Q','A','Q','Q','M','N','Q',
        'K','D','F','L','S','L','I','V','S','I','L','R','S','W','N','E','P','L','Y','H','L',
        'V','T','E','V','R','G','M','Q','E','A','P','E','A','I','L','S','K','A','V','E','I',
        'E','E','Q','T','K','R','L','L','E','G','M','E','L','I','V','S','Q','V','H','P','E',
        'T','K','E','N','E','I','Y','P','V','W','S','G','L','P','S','L','Q','M','A','D','E',
        'E','S','R','L','S','A','Y','Y','N','L','L','H','C','L','R','R','D','S','H','K','I',
        'D','N','Y','L','K','L','L','K','C','R','I','I','H','N','N','N','C']
seq2 = ['P','I','D','N','Y','L','K','L','L','K','C','R','I','I','H','N','N','N','C']

In [3]:
levenshtein(seq1, seq2)

208

In [4]:
def output_k_homologs(reference_sequence, k):
    proteins = open('proteins.fasta', 'r')
    group = {}
    read = False
    seq = []
    key = ''
    for line in proteins:
        if line[0] == '>':
            read = True
            seq = []
            key = line[line.index('[')+1:line.index(']')]
        elif line == '\n':
            read = False
            group[key] = seq
        elif read:
            seq.extend(list(line[:-2]))
    group[key] = seq
    
    for name in group:
        group[name] = levenshtein(reference_sequence, group[name])
    homologs_df = pd.DataFrame([[key,value] for key,value in group.items()],columns=['proteinName','levenshteinDistance'])
    return homologs_df.sort_values(by = ['levenshteinDistance'])[:k]

In [5]:
output_k_homologs(seq1, 10)

,proteinName,levenshteinDistance
376,Homo sapiens,5
211,Pongo abelii,8
122,Pan paniscus,10
232,Nomascus leucogenys,10
167,Gorilla gorilla gorilla,11
38,Pan troglodytes,11
228,Macaca mulatta,12
203,Mandrillus leucophaeus,14
119,Macaca fascicularis,14
260,Macaca nemestrina,14
